In [1]:
## 📚 1. Setup and Data Loading

import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold
from sklearn.experimental import enable_halving_search_cv # Required for HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV # NEW TOOL!
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, make_scorer
from skopt import BayesSearchCV # Requires: pip install scikit-optimize
from skopt.space import Real, Integer # For defining parameter space

# --- Load the same standard regression dataset for fair comparison ---
from sklearn.datasets import make_regression
X, y = make_regression(
    n_samples=500,
    n_features=10,
    n_informative=5,
    random_state=42
)

mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

print(f"Dataset loaded: {len(X)} samples for regression.")
print("Goal: Compare the efficiency of Bayesian Optimization and Successive Halving.")

Dataset loaded: 500 samples for regression.
Goal: Compare the efficiency of Bayesian Optimization and Successive Halving.


## 🧠 2. Bayesian Optimization (The Smart Search)

Unlike Grid or Random Search, **Bayesian Optimization (BO)** doesn't test blindly. It uses the results of past trials to decide which hyperparameter combination to test next.

* It builds a probabilistic model (a "surrogate") of the objective function (our score).
* It chooses the next point that is likely to be either much better than the current best *or* located in an unexplored area of the search space.

This technique is often the most **sample-efficient** (requires the fewest total model fits).

### 2.1. Defining the Search Space (Distributions)

```python
# Use the same Random Forest Regressor
rf_model_bo = RandomForestRegressor(random_state=42)

# Define the search space using distributions (Integer and Real)
# skopt requires its own syntax for defining the ranges.
search_space_bo = {
    'n_estimators': Integer(50, 200),
    'max_depth': Integer(5, 30),
    'min_samples_split': Integer(2, 20),
    'max_features': ['sqrt', 'log2', 1.0]
}

n_iterations_bo = 40 # Use the same number of iterations as Random Search for comparison
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=42)

# 2.2. Initialize and Run Bayesian Optimization
bo_search = BayesSearchCV(
    estimator=rf_model_bo,
    search_spaces=search_space_bo,
    n_iter=n_iterations_bo,
    scoring=mae_scorer,
    cv=cv_strategy,
    random_state=42,
    verbose=0,
    n_jobs=-1
)

print(f"\nStarting Bayesian Optimization ({n_iterations_bo} iterations)...")
start_time_bo = time.time()
bo_search.fit(X, y)
end_time_bo = time.time()
bo_time = end_time_bo - start_time_bo

In [2]:
# Use the same Random Forest Regressor
rf_model_bo = RandomForestRegressor(random_state=42)

# Define the search space using distributions (Integer and Real)
# skopt requires its own syntax for defining the ranges.
search_space_bo = {
    'n_estimators': Integer(50, 200),
    'max_depth': Integer(5, 30),
    'min_samples_split': Integer(2, 20),
    'max_features': ['sqrt', 'log2', 1.0]
}

n_iterations_bo = 40 # Use the same number of iterations as Random Search for comparison
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=42)

# 2.2. Initialize and Run Bayesian Optimization
bo_search = BayesSearchCV(
    estimator=rf_model_bo,
    search_spaces=search_space_bo,
    n_iter=n_iterations_bo,
    scoring=mae_scorer,
    cv=cv_strategy,
    random_state=42,
    verbose=0,
    n_jobs=-1
)

print(f"\nStarting Bayesian Optimization ({n_iterations_bo} iterations)...")
start_time_bo = time.time()
bo_search.fit(X, y)
end_time_bo = time.time()
bo_time = end_time_bo - start_time_bo


Starting Bayesian Optimization (40 iterations)...


c:\f5\CrossValidationAndHyperparameterTuningTraining\venv\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(18), np.str_('sqrt'), np.int64(3), np.int64(200)] before, using random point [np.int64(10), 'log2', np.int64(20), np.int64(72)]
  warnings.warn(
c:\f5\CrossValidationAndHyperparameterTuningTraining\venv\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(18), np.str_('sqrt'), np.int64(3), np.int64(200)] before, using random point [np.int64(14), 'log2', np.int64(12), np.int64(68)]
  warnings.warn(


#### 🧠 Why the Warnings Appear

**That is a common and important warning when using Bayesian Optimization with the `scikit-optimize` library (`skopt`).**

**The warnings you are seeing:**

**UserWarning: The objective has been evaluated at point \[...\] before, using random point \[...\]**

**occur because the Bayesian optimization process is trying to select the most *promising* next point to evaluate, but it has accidentally selected a hyperparameter combination that it has already tested in a previous iteration.**

**Here's a breakdown of why this happens and why `skopt` switches to a random point:**

---

## **🧠 Why the Warnings Appear in `BayesSearchCV`**

**Bayesian Optimization works by balancing exploration (trying new, unknown regions of the parameter space) and exploitation (testing near the best-performing points found so far).**

### **1\. The Core Mechanism**

1. **Surrogate Model: `skopt` builds a Gaussian Process model (the surrogate) to estimate the model's performance across the entire search space.**  
2. **Acquisition Function: It uses this surrogate model to find the next point that maximizes the Expected Improvement (EI)—the point where the score is likely to be best.**

### **2\. The Collision (The Warning)**

**Due to the nature of continuous sampling (especially when using `Integer` ranges and a limited number of unique candidates):**

* **The acquisition function may calculate that the optimal next point is a set of hyperparameters (e.g., `max_depth=18`, `n_estimators=200`) that was already tested in a previous iteration.**  
* **Since re-evaluating an already-tested point is a waste of time and computational resources, `skopt` uses a guardrail.**

### **3\. The `skopt` Fallback (The Resolution)**

* **When `skopt` detects that the chosen "optimal" point is a duplicate, it discards that point and instead selects a new, purely random point from the search space to continue the process.**  
* **This ensures the tuning process moves forward and doesn't get stuck in a loop.**

## **📝 Is This a Problem?**

**No, this is generally NOT a problem for the final result of your tuning.**

1. **The tuning process is still valid: `skopt` intelligently avoids wasted time by switching to an untested point.**  
2. **It's common: This happens frequently, especially when the search space is limited or when the best hyperparameters are clustered together.**

**You can safely ignore these warnings. They are simply information about the internal decision-making process of the Bayesian optimizer.**



## 🏆 3. Successive Halving (The Tournament)

**Successive Halving** is a method that treats hyperparameter candidates like competitors in a tournament:

1.  **Round 1:** Test *many* candidates on a **small subset** of the training data (e.g., 20% of samples).
2.  **Elimination:** Discard the candidates that performed poorly (e.g., keep the top 50%).
3.  **Next Round:** Test the remaining candidates on a **larger subset** of the data.
4.  **Final Round:** Only the top few candidates are tested on the **full dataset**.

This saves massive amounts of time by only fully training the most promising candidates. We use `HalvingRandomSearchCV` here.

### 3.1. Defining the Halving Search

```python
# Use the same Random Forest Regressor and search space distributions
rf_model_hs = RandomForestRegressor(random_state=42)
param_distribution_hs = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20, 30, None],
    'min_samples_split': [2, 5, 10, 20],
    'max_features': ['sqrt', 'log2', 1.0]
}

# 3.2. Initialize and Run Halving Random Search
# factor=2 means we discard half of the candidates at each round.
hs_search = HalvingRandomSearchCV(
    estimator=rf_model_hs,
    param_distributions=param_distribution_hs,
    factor=2, # Halve candidates/double resources in each round
    n_candidates=40, # Start with 40 random candidates
    scoring=mae_scorer,
    cv=cv_strategy,
    random_state=42,
    verbose=1,
    n_jobs=-1
)

print(f"\nStarting Successive Halving (starts with 40 candidates)...")
start_time_hs = time.time()
hs_search.fit(X, y)
end_time_hs = time.time()
hs_time = end_time_hs - start_time_hs

In [3]:
# Use the same Random Forest Regressor and search space distributions
rf_model_hs = RandomForestRegressor(random_state=42)
param_distribution_hs = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20, 30, None],
    'min_samples_split': [2, 5, 10, 20],
    'max_features': ['sqrt', 'log2', 1.0]
}

# 3.2. Initialize and Run Halving Random Search
# factor=2 means we discard half of the candidates at each round.
hs_search = HalvingRandomSearchCV(
    estimator=rf_model_hs,
    param_distributions=param_distribution_hs,
    factor=2, # Halve candidates/double resources in each round
    n_candidates=40, # Start with 40 random candidates
    scoring=mae_scorer,
    cv=cv_strategy,
    random_state=42,
    verbose=1,
    n_jobs=-1
)

print(f"\nStarting Successive Halving (starts with 40 candidates)...")
start_time_hs = time.time()
hs_search.fit(X, y)
end_time_hs = time.time()
hs_time = end_time_hs - start_time_hs


Starting Successive Halving (starts with 40 candidates)...
n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 10
max_resources_: 500
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 40
n_resources: 10
Fitting 5 folds for each of 40 candidates, totalling 200 fits
----------
iter: 1
n_candidates: 20
n_resources: 20
Fitting 5 folds for each of 20 candidates, totalling 100 fits
----------
iter: 2
n_candidates: 10
n_resources: 40
Fitting 5 folds for each of 10 candidates, totalling 50 fits
----------
iter: 3
n_candidates: 5
n_resources: 80
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 4
n_candidates: 3
n_resources: 160
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 5
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits


In [4]:
## 📈 4. Final Comparison and Conclusion

# Assuming Random Search time and score from Notebook 02 are available for comparison

# Display results from this notebook
print("\n--- Bayesian Optimization Results ---")
print(f"Time Taken: {bo_time:.2f} seconds.")
print(f"Best CV Score (MAE): {-bo_search.best_score_:.3f}")
print(f"Best Parameters: {bo_search.best_params_}")

print("\n--- Successive Halving Results ---")
print(f"Time Taken: {hs_time:.2f} seconds.")
print(f"Best CV Score (MAE): {-hs_search.best_score_:.3f}")
print(f"Best Parameters: {hs_search.best_params_}")

# --- CONCLUSION ---

print("\n\n--- Conclusion and Rationale ---")


--- Bayesian Optimization Results ---
Time Taken: 188.65 seconds.
Best CV Score (MAE): 18.829
Best Parameters: OrderedDict({'max_depth': 18, 'max_features': 1.0, 'min_samples_split': 2, 'n_estimators': 149})

--- Successive Halving Results ---
Time Taken: 49.60 seconds.
Best CV Score (MAE): 21.276
Best Parameters: {'n_estimators': 50, 'min_samples_split': 2, 'max_features': 1.0, 'max_depth': 30}


--- Conclusion and Rationale ---


## 🌟 5. Conclusion and Dataset Rationale

### Summary of Tuning Methods

| Method | Approach | Speed | Score | Best Use Case |
| :--- | :--- | :--- | :--- | :--- |
| **Random Search** | Tests random points. | Fast | High (Good) | General, initial tuning. |
| **Bayesian Opt.** | Tests smart points (learns as it goes). | Medium | Highest (Best) | Finding the absolute optimum in complex spaces. |
| **Successive Halving** | Tournament (eliminates poor candidates early). | Fastest | High (Good) | Very large search spaces where speed is critical. |

### 🎯 Rationale for Dataset Choice (Why Not My Supplement Data?)

* **Goal of Notebooks 01-03:** The core objective was to show the **time efficiency** and **methodology** of Grid, Random, and Advanced Searches.
* **The Problem:** Tuning with the `Supplement_Sales_Weekly_Expanded.csv` data requires the **TimeSeriesSplit** CV strategy. **TimeSeriesSplit is computationally slow** because it must retrain the model on an ever-expanding subset of data in *every* fold.
* **The Result:** If we used the TimeSeries data, the notebook would take a very long time to run, and we wouldn't be able to clearly demonstrate the speed advantage of Random Search, Bayesian Optimization, or Halving against each other.
